In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing

%matplotlib inline

In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 5.5MB/s eta 0:00:01                   | 1.4MB 5.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install geocoder
!pip install geopy

     |████████████████████████████████| 102kB 5.6MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.0MB/s  eta 0:00:01
     |████████████████████████████████| 829kB 6.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01


In [5]:
# Use Pandas to load html data
pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Create dataframe showing Toronto Neighborhoods by using pandas
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=df[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# drop cells with a borough that is Not assigned
index_NA=df[(df['Borough']=='Not assigned')].index
df.drop(index_NA, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# If a cell has a borough but a Not assigned neighborhood, set the neighborhood as the borough
df.loc[df.Neighborhood=='Not assigned','Neighborhood']=df.Neighborhood
df.shape

(103, 3)

In [11]:
#use pandas to read csv file that has the geographical coordinates of each postal code to a dataframe
df_coordinators=pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinators.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Create dataframe df_CA of Toronto Neighborhood by merging two dataframes:df & df_coordinators
df_CA = pd.merge(left=df,right=df_coordinators,on='Postal Code')
df_CA.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [13]:
#Check numbers of Boroughs and Neighborhoods in the df_CA
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_CA['Borough'].unique()),
        df_CA.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [14]:
# Import more libraries
import json #handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim #module to convert an address into latitude and longitude
import requests #library to handle request
from pandas.io.json import json_normalize #transform json file to pandas df

!conda install -c conda-forge folium=0.5.0 --yes
import folium #plotting library

#Matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# K-means from clustering stage
from sklearn.cluster import KMeans


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [16]:
# Define user_agent, name it to_explorer
address='Toronto, ON'

geolocator = Nominatim(user_agent='to_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print ('The geographical coordinate of Toronto are {},{}.'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.6534817,-79.3839347.


In [20]:
#Create map of Toronto using Latitude and Longitude values
map_Toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_CA['Latitude'],df_CA['Longitude'],df_CA['Borough'],df_CA['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [21]:
# We only cluster the neighborhoods in Downtown Toronto, so slice the df to create a new df for downtown toronto
df_dto=df_CA[df_CA['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_dto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [22]:
# Get geographical coordinates of Downtown Toronto
# Define user_agent, name it dto_explorer
address='Downtown Toronto, ON'

geolocator = Nominatim(user_agent='dto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print ('The geographical coordinate of Toronto are {},{}.'.format(latitude,longitude))

The geographical coordinate of Toronto are 43.6563221,-79.3809161.


In [27]:
#Create map for downtown toronto
map_DowntownToronto = folium.Map(location=[latitude,longitude], zoom_start=13)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_dto['Latitude'],df_dto['Longitude'],df_dto['Borough'],df_dto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DowntownToronto)
    
map_DowntownToronto

In [28]:
# Define Foursquare Credentials
CLIENT_ID = 'your-client ID'
CLIENT_SECRET = 'your-client-secret'
VERSION = '20180605'

In [32]:
# Explore the first neighborhood in df
# Get the neighborhood's latitude & Longitude
neighborhood_name=df_dto.loc[0,'Neighborhood']
neighborhood_lat=df_dto.loc[0,'Latitude']
neighborhood_lng=df_dto.loc[0,'Longitude']

print ('Neighborhood name: {}; Latitude:{}; Longitude:{}'.format(neighborhood_name,
                                                                 neighborhood_lat,
                                                                 neighborhood_lng))

Neighborhood name: Regent Park, Harbourfront; Latitude:43.6542599; Longitude:-79.3606359


In [35]:
# Get top 100 venues in Regent Part, Harbourfront within radius of 800 meters
LIMIT = 100
radius = 800
#create URL
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_lat,
    neighborhood_lng,
    radius,
    LIMIT)
url#display url


'https://api.foursquare.com/v2/venues/explore?&client_id=JIVUD0KYY0XAUUWMOLZVVZU10EOS2IFCNQBFSQ1AIOJOIXJ4&client_secret=GV5F13XPZW0YS1D04GEQH00E1PM3DVZQK2RLBOKJJV3TL3CT&v=20180605&ll=43.6542599,-79.3606359&radius=800&limit=100'

In [36]:
#send the GET request and get the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0001ee3395b26d6054ce02'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 81,
  'suggestedBounds': {'ne': {'lat': 43.661459907200005,
    'lng': -79.35070310023775},
   'sw': {'lat': 43.647059892799994, 'lng': -79.37056869976226}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label

In [37]:
# function extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
# Clean the json and structure it to pd df
venues = results['response']['groups'][0]['items']

nearby_venue = json_normalize(venues)

#filter columns
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venue.loc[:,filtered_columns]

#filter categories
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Corktown Common,Park,43.655618,-79.356211
4,The Distillery Historic District,Historic Site,43.650244,-79.359323
